<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/TwADR_bigtweet_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-1.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-1.validation.csv",header=None)
valid.head()

,0,1
0,85,tremor
1,119,anxiety
2,154,hives
3,189,dizziness
4,189,dizziness


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-1.test.csv",header=None)
test.head()

,0,1
0,13,Insomnia
1,13,awake in the middle of the night
2,61,very stupid
3,133,night terrors
4,189,dizziness


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR-bigtweet.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'to',
 'the',
 'in',
 'rt',
 'of',
 'for',
 'a',
 'you',
 'i',
 'on',
 'is',
 'disease',
 'pain',
 'and',
 'my',
 'mood',
 'me',
 'amp',
 'increased',
 'with',
 'gluten',
 ',',
 'disorder',
 'withdrawal',
 'abnormal',
 'anxiety',
 'your',
 'it',
 'symptoms',
 'free',
 'have',
 'weight',
 'decreased',
 'out',
 'are',
 'this',
 'new',
 'from',
 'like',
 'not',
 'about',
 'disorders',
 'up',
 '-',
 '(',
 ')',
 'our',
 'do',
 'glutenfree',
 'sleeplessness',
 'what',
 'be',
 'drug',
 'behavior',
 'nt',
 'syndrome',
 'celiac',
 'feeling',
 'now',
 'got',
 'diseases',
 'at',
 'or',
 'somnolence',
 'insomnia',
 'sleep',
 'blood',
 'disturbance',
 'finding',
 'that',
 'so',
 'all',
 'right',
 'energy',
 'food',
 'how',
 'can',
 'mental',
 'we',
 'skin',
 'more',
 'acute',
 'depression',
 'via',
 'gain',
 'appetite',
 'its',
 'feel',
 'body',
 'tip',
 'excitability',
 'nausea',
 'if',
 'as',
 'ceiling',
 '

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.366084,3.542057,0.448605,08:15


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.282768,3.033051,0.505078,08:14


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.037021,2.986022,0.509975,08:14


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.979152,2.946609,0.514495,08:14


In [25]:
learn.save_encoder('TwADR-bigtweet_fold1_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR-bigtweet_fold1_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.204280,6.642648,0.114035,00:21


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.308655,5.187312,0.166667,00:25


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.747407,4.699819,0.192982,01:00


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.467620,4.201406,0.236842,00:58
1,5.241755,4.109206,0.254386,00:56


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.001527,3.874995,0.254386,00:57
1,4.807856,3.676269,0.289474,00:56


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.602865,3.609183,0.289474,00:56
1,4.415514,3.541717,0.350877,00:56


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.223264,3.330058,0.385965,00:56
1,4.081351,3.221958,0.368421,01:01


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.886884,3.130270,0.350877,00:56
1,3.759220,3.065353,0.359649,01:03


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.613041,3.037415,0.333333,00:59
1,3.516145,2.968035,0.385965,00:56
2,3.324007,2.929673,0.412281,00:56
3,3.156744,2.874431,0.421053,00:58


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.016948,2.849046,0.438596,00:58
1,2.957824,2.861503,0.438596,00:57
2,2.766339,2.834181,0.429825,00:56
3,2.638739,2.787225,0.438596,01:01


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.470618,2.767357,0.429825,00:59
1,2.423333,2.806352,0.447368,00:59
2,2.275543,2.759206,0.438596,01:01
3,2.100828,2.764104,0.438596,00:58


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.983026,2.714424,0.438596,00:58
1,1.971360,2.805277,0.464912,01:01
2,1.850850,2.803286,0.464912,00:57
3,1.685297,2.774911,0.473684,01:01


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.566243,2.773621,0.421053,00:59
1,1.570270,2.777967,0.412281,00:58
2,1.448795,2.769996,0.464912,01:02
3,1.325171,2.806901,0.447368,00:59


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.249287,2.793453,0.456140,00:58
1,1.245757,2.859190,0.473684,01:03
2,1.158180,2.782294,0.508772,00:59
3,1.048397,2.782950,0.473684,01:00


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.994099,2.751231,0.491228,00:57
1,1.017857,2.876628,0.447368,01:02
2,0.948382,2.879220,0.456140,01:01
3,0.889844,2.910886,0.456140,00:58


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.838374,3.007973,0.447368,01:00
1,0.821121,3.005362,0.429825,00:58


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.746964,3.055641,0.473684,00:57
1,0.703456,3.064703,0.456140,00:59


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.709283,3.014019,0.456140,01:01


In [44]:
# save the best model

learn.save_encoder('TwADR-bigtweet_fold1')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.638105,3.108320,0.447368,00:59


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.677691,3.082375,0.482456,01:01


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.606041,3.102995,0.456140,01:00


In [51]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.584759,3.247916,0.421053,00:59


# Part three: Predict on the test dataset

In [52]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [53]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[393, 13, 792, 133, 189, 189, 792, 2147, 1936, 2147, 1787, 2032, 1787, 1136, 418, 1073, 441, 744, 1936, 97, 791, 1597, 1458, 224, 974, 818, 233, 825, 1884, 233, 233, 974, 974, 792, 974, 974, 974, 1936, 978, 13, 1936, 978, 1062, 109, 1767, 1089, 1295, 1787, 1175, 1531, 1019, 1546, 1099, 1295, 1295, 1610, 1295, 1295, 1295, 2074, 1319, 233, 755, 1432, 1432, 1125, 1529, 1529, 109, 233, 2074, 966, 1594, 1597, 1769, 1597, 392, 392, 1610, 1610, 233, 1526, 1145, 233, 1073, 1787, 792, 1531, 1531, 233, 233, 1767, 1767, 1798, 1767, 1718, 233, 1529, 1099, 2032, 2032, 2032, 2147, 2023, 923, 1597, 1099, 1109, 1099, 233, 1936, 1936, 1936, 1340, 1936, 233, 1295, 1767, 1295, 772, 2103, 1813, 1767, 2032, 1936, 1787, 189, 189, 233, 231, 2081, 2147, 2147, 233, 2147, 13, 2147, 233, 233, 824, 2191, 1529, 1521]
[13, 13, 61, 133, 189, 189, 195, 233, 233, 233, 393, 393, 393, 396, 418, 452, 561, 744, 749, 750, 791, 791, 809, 813, 813, 818, 824, 824, 825, 852, 947, 974, 974, 974, 974, 974, 974, 978, 978,

In [54]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

44
0.3076923076923077
